[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/Red_Recurrente_Forward.ipynb)

# Datos a utilizar

Datos originales: https://challengedata.ens.fr/participants/challenges/31/

El problema trata de buscar un algoritmo de clasificación que ayude a crear estrategias de inversión en criptomonedas, basado en el "sentimiento" extraído de noticias y redes sociales.

Por cada hora de trading se contabilizó la ocurrencia de algunos terminos, tales como 'adoption' y 'hack', en un selecto numero de cuentas influyentes de twitter y en algunos foros como 'Bitcointalk'.

Se han creado 10 temas diferentes, algunos positivos y otros negativos y se han contabilizado las palabras antes mencionadas, antes de una normalización.

Dado un tema y un tema, hemos visto los conteos de las últimas 48 horas y se estandarizaron esos conteos. El resultado se multiplicó por el conteo promedio por hora y se dividió por el conteo promedio por hora de todo el entrenamiento

Se agregaron 5 características correspondientes a los precios finales en periodos de 1 hr, 6 hrs, 12 hrs, 24 hrs y 48 hrs
El objetivo es predecir si el precio del Bitcoin tendrá un retorno (en la próxima hora) que sea de mas del 0.2%, entre -0.2% y 0.2% o menos al -0.2%.

La métrica utilizada para la perdida es la perdida logistica, definita como el negativo de la log-verosimilitud de las etiquetas verdaderas comparadas con las probabilidades predichas por el clasificador.

Las verdaderas etiquetas están codificadas como una matríz de 3 columnas, donde hay unos o ceros dependiendo si el elemento pertenece a la categoría de una columna u otra.
 
Dada una matriz P de probabilidades $p_{i,k}=Pr(t_{i,k}=1)'$ , la función de perdida se define como

$$
L_{log}(Y,P)=-log{Pr(Y|P)}=-\frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^3{y_{i,k}log(p_{i,k})}
$$

Entre más bajo el score de ésta medida, mejor.



Resumen datos: 

- ***ID***: Id de la muestra, que está ligado con el ID del archivo de variables objetivo.

- ***I_1_lag(k)*** hasta ***I_10_lag(k)***: Valores de indicadores para cada $k$ de "lag" ($k\in[\![0;47]\!]$) representando valores normalizados de los indicadores *I_1* al *I_10* cada hora en las últimas 48 horas.

- ***X_1*** to ***X_5***: Valores de 5 indicadores normalizados que representan la variación de precio de Bitcoin en las ultimas 1, 6, 12, 24, y 48 horas.

Los *outputs* o salidas, están en un archivo .csv. cada linea corresponde a una muestra, donde:

- ***ID***: Id de la muestra.
- ***Target_-1***: Clasificación del retorno del Bitcoin en la próxima hora. $-1$ representa un movimiento hacia abajo menor al $-0.2\%$.
- ***Target_0***: Clasificación del retorno del Bitcoin en la próxima hora. $0$ representa un movimiento entre $-0.2\%$ y $0.2\%$.
- ***Target_1***: Clasificación del retorno del Bitcoin en la próxima hora. $1$ representa un movimiento hacia arriba mayor a $0.2\%$.



# Dependencias

In [ ]:
import pandas as pd
import numpy as np

Viejas conocidas, nos serán de utilidad

In [ ]:
def linear(x):
    return x


def relu(x):
    return np.where(x > 0, x, 0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_raw = pd.read_csv('/content/drive/MyDrive/Curso-NN-de-Cero/semana5/input_training_IrTAw7w.csv').set_index('ID')
X_raw

,X1,X2,X3,X4,X5,I1_lag0,I1_lag1,I1_lag2,I1_lag3,I1_lag4,...,I10_lag38,I10_lag39,I10_lag40,I10_lag41,I10_lag42,I10_lag43,I10_lag44,I10_lag45,I10_lag46,I10_lag47
ID,,,,,,,,,,,,,,,,,,,,,
0,0.460020,0.620360,-0.972192,2.745197,4.177783,2.325865,2.060138,0.071162,2.360597,-0.611526,...,-0.342912,-0.194165,0.122331,0.028682,-0.093626,-0.559840,0.562584,-0.557868,1.424906,-0.016294
1,-0.347872,-2.199925,-0.222026,3.741888,8.608291,-4.091293,-3.502499,-1.463631,0.383153,-3.669962,...,1.261341,-0.082428,-1.035813,-0.249607,-0.971215,-0.058408,1.460632,-0.653394,-1.743487,4.065305
2,-2.152963,-0.432461,1.619057,-0.003912,3.870262,-0.598858,-0.412391,-0.765354,-0.998152,-0.938755,...,2.245204,3.002347,2.674186,2.656251,1.062974,-0.484619,-0.044594,1.579731,0.962836,1.146983
3,-1.827669,-1.881770,-4.214322,0.178225,0.992362,0.383757,2.512478,-0.383434,-0.208506,-1.104289,...,1.383203,-1.338892,0.298076,1.808275,2.837975,2.054112,0.741138,1.701911,0.110082,0.114980
4,0.748761,1.799939,1.561006,5.204120,2.161637,-1.275226,-1.544131,-1.802590,-1.128526,-0.469835,...,-0.477313,0.742923,-0.273225,1.311015,0.744330,2.914322,1.030602,0.480722,-0.492838,1.377958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,-0.074522,-0.472044,-2.860659,-1.266230,-10.229167,0.583145,-0.051301,-0.584659,1.458945,2.004759,...,-1.634290,-1.527111,-1.008016,-0.500519,1.277636,1.257714,0.502732,1.751844,0.150679,-0.533808
13996,1.730118,3.177408,0.816198,1.136877,-1.588960,1.011735,-0.185748,-0.522647,2.316802,1.219339,...,-0.409750,0.840944,-1.804313,0.357944,-1.058557,-0.196874,-2.507582,0.125756,1.532976,-1.087343
13997,2.093028,4.108092,1.056253,8.163642,8.916299,2.338713,2.554397,1.665492,3.719985,-0.278893,...,-0.531223,-1.249847,-1.288419,-0.897649,-0.199824,-0.033545,0.240647,2.188396,0.039340,0.756515


In [ ]:
y_raw = pd.read_csv('/content/drive/MyDrive/Curso-NN-de-Cero/semana5/output_training_F2dZW38.csv').set_index('ID')
y_raw

,Target -1,Target 0,Target 1
ID,,,
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,1,0
...,...,...,...
13995,0,0,1
13996,0,1,0
13997,0,1,0


# Escoger datos a utilizar

Nos quedaremos con los valores relacionados a los sentimientos de las notas extraídas para 2 temas, y la columna de la variación del bitcoin cada hora.

Tomaremos $p=5$ (tiempo que miraremos hacia atrás) y $n=7$ (tamaño de la capa oculta) y $3$ características a ver, los valores de $I1$ e $I2$ y los precios $Xi$

In [ ]:
features_1 = ['I1_lag47', 'I1_lag23', 'I1_lag11', 'I1_lag5', 'I1_lag0']

In [ ]:
features_2 = ['I2_lag47', 'I2_lag23', 'I2_lag11', 'I2_lag5', 'I2_lag0']

In [ ]:
precios = ['X5', 'X4', 'X3', 'X2', 'X1']

In [ ]:
X_selecto = X_raw[precios + features_1 + features_2]
X_selecto

,X5,X4,X3,X2,X1,I1_lag47,I1_lag23,I1_lag11,I1_lag5,I1_lag0,I2_lag47,I2_lag23,I2_lag11,I2_lag5,I2_lag0
ID,,,,,,,,,,,,,,,
0,4.177783,2.745197,-0.972192,0.620360,0.460020,0.123301,-0.303876,-1.570320,-0.474029,2.325865,0.084599,1.093088,-0.336963,-0.364099,0.847821
1,8.608291,3.741888,-0.222026,-2.199925,-0.347872,1.389198,-3.016570,1.469743,-2.118337,-4.091293,-0.389125,-1.219913,0.740208,0.779588,-2.032579
2,3.870262,-0.003912,1.619057,-0.432461,-2.152963,1.560763,2.071983,-0.935591,-0.890084,-0.598858,0.948492,0.865920,-0.717741,-0.512224,-0.171448
3,0.992362,0.178225,-4.214322,-1.881770,-1.827669,0.766485,3.888724,-2.562135,-1.055666,0.383757,-0.570012,-0.102196,-0.354586,-0.647492,3.078638
4,2.161637,5.204120,1.561006,1.799939,0.748761,2.419920,0.351939,-1.497806,-1.033174,-1.275226,1.860046,2.108902,-0.088636,-1.198800,-1.419191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,-10.229167,-1.266230,-2.860659,-0.472044,-0.074522,1.212270,-0.326625,-1.044508,-0.188567,0.583145,0.325598,1.759328,0.045704,0.693354,0.456789
13996,-1.588960,1.136877,0.816198,3.177408,1.730118,0.627397,0.213728,3.681717,-0.112632,1.011735,0.892681,-0.298030,2.520059,3.283997,1.779191
13997,8.916299,8.163642,1.056253,4.108092,2.093028,-0.430326,1.587833,2.317140,-0.313868,2.338713,-0.704965,2.820523,-0.755655,-0.106609,0.575855


Hay que reacomodar los datos, para tener pequeñas series de tiempo que tengan 5 tiempos distintos.

In [ ]:
data_array = np.zeros((len(X_selecto), 5, 3))

In [ ]:
for i in range(len(X_selecto)):
    data_array[i] = X_selecto.iloc[i, :].values.reshape(3,5).T
    print(data_array[i])
    break

[[ 4.17778324  0.12330096  0.08459858]
 [ 2.74519725 -0.30387588  1.09308833]
 [-0.97219231 -1.57032042 -0.33696346]
 [ 0.62035968 -0.47402934 -0.36409929]
 [ 0.46002033  2.3258649   0.84782066]]


In [ ]:
for i in range(len(X_selecto)):
    data_array[i] = X_selecto.iloc[i, :].values.reshape(3,5).T

In [ ]:
data_array.shape

(14000, 5, 3)

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# Un arreglo con series temporales para 3 características (las filas representan como va avanzando el tiempo)
data_array

array([[[ 4.17778324,  0.12330096,  0.08459858],
        [ 2.74519725, -0.30387588,  1.09308833],
        [-0.97219231, -1.57032042, -0.33696346],
        [ 0.62035968, -0.47402934, -0.36409929],
        [ 0.46002033,  2.3258649 ,  0.84782066]],

       [[ 8.60829058,  1.38919769, -0.38912495],
        [ 3.74188805, -3.01657043, -1.21991286],
        [-0.22202592,  1.46974303,  0.74020831],
        [-2.19992478, -2.11833719,  0.77958786],
        [-0.34787206, -4.091293  , -2.03257946]],

       [[ 3.87026224,  1.56076271,  0.94849226],
        [-0.00391238,  2.07198349,  0.8659197 ],
        [ 1.61905739, -0.93559074, -0.717741  ],
        [-0.43246086, -0.89008449, -0.5122244 ],
        [-2.15296262, -0.59885808, -0.17144772]],

       ...,

       [[ 8.91629887, -0.43032551, -0.70496467],
        [ 8.16364221,  1.58783271,  2.8205228 ],
        [ 1.05625294,  2.31714006, -0.75565513],
        [ 4.10809178, -0.31386772, -0.1066092 ],
        [ 2.09302801,  2.33871299,  0.57585518]],


Tomemos solo uno de los datos (el primero) para ejemplificar como avanzaría la red

In [ ]:
seriesita = data_array[0]
seriesita

array([[ 4.17778324,  0.12330096,  0.08459858],
       [ 2.74519725, -0.30387588,  1.09308833],
       [-0.97219231, -1.57032042, -0.33696346],
       [ 0.62035968, -0.47402934, -0.36409929],
       [ 0.46002033,  2.3258649 ,  0.84782066]])

# Veamos como se alimentaría la red para un solo valor (despues pueden ciclar o decidir si cambian el estilo de la multiplicación de matrices).

La imagen muestra como es nuestro modelo muy generalmente
<center> <img src='https://github.com/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/images/red.png?raw=true' width=300> </center>

Primero, en la imagen de abajo, veamos que debemos definir (estado oculto $H_0$, matrices de pesos y de sesgos que tendrá la neurona). La imagen muestra como sería la entrada del primer dato de la serie de tiempo y que saldría de la neurona.

<center> <img src='https://github.com/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/images/t_1.png?raw=true' width=500> </center>

**NOTA:** El vector de sesgos es en realidad $B_h$, que afecta a los estados ocultos, ya que despues definiremos un $B$ con el cual no debe confundirse

In [ ]:
# Primer estado oculto "H_0", arriba dijimos que "n" sería igual a 6
H_0 = np.random.randn(1,7)
H_0

array([[ 2.00080169,  0.25148641, -1.47785606, -1.30892261, -0.52856977,
         1.46196243,  0.42961185]])

In [ ]:
# Pesos que hacen un producto punto con los datos de entrada de la serie de tiempo
W_x = np.random.randn(3,7)
W_x

array([[ 1.06066247,  1.72275996,  0.72074399, -0.33071617,  0.03798293,
        -1.52952405,  1.59448278],
       [ 1.7842374 ,  0.98211354, -1.10566884,  0.19595719,  2.21485583,
         0.72285781, -0.28323368],
       [-1.00889885, -0.9228497 ,  2.05059104, -1.03444399,  0.76469578,
         0.15501751,  1.27261755]])

In [ ]:
# Pesos que multiplican el estado oculto anterior para generar una parte del nuevo estado oculto
W_h = np.random.randn(7,7)
W_h

array([[ 0.93769511, -1.34384051,  0.32189843, -1.89521802, -0.94468624,
        -1.44218519, -0.64230056],
       [-0.52189844, -0.71020496, -0.79132225, -1.45813751, -0.06593407,
        -0.93772038, -1.42063259],
       [ 1.7771382 ,  0.08262092,  0.34452191,  0.62627867,  0.33400386,
        -0.12069739, -0.60106045],
       [ 1.00197004,  0.94835284,  0.1649459 ,  0.96108167,  0.73813376,
         0.16172797,  0.11530825],
       [ 0.98031069,  1.28680378,  0.25157899, -0.82350378,  0.02821466,
         0.24293452, -1.0220836 ],
       [-1.05619823, -0.06445143, -0.04242132, -0.77463845, -0.39817197,
         0.50284577,  0.45470089],
       [-0.0443153 , -1.22729398, -1.39644938,  0.09372488,  0.14245357,
         1.8861764 , -1.34450456]])

In [ ]:
# Sesgo para generar una parte del estado oculto
B_h = np.random.rand(1,7)
B_h

array([[0.96312163, 0.8100341 , 0.03787259, 0.15918599, 0.63046225,
        0.28560728, 0.44817915]])

## Recuerden que $X$ es toda la serie, entonces $X_1$ con el que alimentaremos en el primer paso es el primer renglón de $X$ (variable llamada "seriesita" [de serie pequeña])

In [ ]:
# ¿Con que secuencia de código obtendrían X_1?
X_1 = seriesita[0]
X_1

array([4.17778324, 0.12330096, 0.08459858])

In [ ]:
# Según la imagen, la operación a realizar será la siguiente
H_1 = sigmoid(np.dot(X_1,W_x) + H_0 @ W_h + B_h)
H_1

array([[0.77811224, 0.92538973, 0.88196781, 0.00025222, 0.0586695 ,
        0.00043498, 0.99899791]])

# EJERCICIOS:



# 1. En base a la imagen de abajo, ¿como obtendria $H_2$? Realice los calculos

<center> <img src='https://github.com/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/images/t_2.png?raw=true' width=500> </center>

In [ ]:
# Espacio para solución de Ejercicio 1
X_2 = seriesita[2]
H_2 = sigmoid(np.dot(X_2,W_x) + H_1 @ W_h + B_h)

# 2. Realice los pasos para obtener hasta $H_4$

In [ ]:
# Espacio para solución de ejercicio 2
X_3 = seriesita[2]
H_3 = sigmoid(np.dot(X_3,W_x) + H_2 @ W_h + B_h)

X_4 = seriesita[3]
H_4 = sigmoid(np.dot(X_4,W_x) + H_3 @ W_h + B_h)

## Forma general como iteramos en la capa (salvo la última etapa)

<center> <img src='https://github.com/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/images/t_general.png?raw=true' width=500> </cemter>

# 3. Según la imagen posterior, vemos como obtendremos la imagen de salida, tras que el profesor la explique, tratar de implementar las operaciones para obtener su salida $\hat{Y}$

<center> <img src='https://github.com/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana5/images/ultima.png?raw=true' width=500> </center>

In [ ]:
def softmax(X):
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

In [ ]:
# Espacio para ejercicio 3
X_5 = seriesita[4]
H_5 = sigmoid(np.dot(X_5,W_x) + H_4 @ W_h + B_h)

W = np.random.rand(7,3)
B = np.random.rand(1,3)

In [ ]:
salida = softmax(H_5 @ W + B)
salida

array([[0.55006502, 0.17225529, 0.27767969]])

1. ¿Cómo interpretamos la salida de esta red (es una red recurrente simple)?

2. ¿Podriamos tratar de resolver este problema con otro algoritmo visto? ¿Que solución propondrían?

